![alt text](sherpa.ai.png "Sherpa.ai")
# Custom Content API Tutorial
The goal of this tutorial is to provide detailed, step-by-step instructions to build the minimal structure for a recommender system. The examples provided in this tutorial are based on the dataset [Data Science for Good: DonorsChoose.org](https://www.kaggle.com/donorschoose/io).

We are going to build a system that recommends projects to donors with similar interests, based on previous donations.

### Nota Bene
In order to improve the readability of this tutorial, a helper `SherpaRequest` has been created, where the headers required to authenticate the API user are set. Please, refer to the [documentation](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/api-reference/authentication) for the corresponding instructions.

## Setup the helper with your keys
You can find them in your account information in the private area: https://developers.sherpa.ai/account/.

In [ ]:
import sherpa_request
from sherpa_request import *

# Setup the request helper
sherpa_request.sherpa_api_key = "YOUR_API_KEY"
sherpa_request.sherpa_private_key = "YOUR_PRIVATE_KEY"

## Building a Recommender System
### Create a Catalog
This section shows how to build the structure of an item catalog. In this case, we have a CSV file containing the following information about projects from DonorsChoose:

Field | Description
--- | ---
`itemId` | A string that uniquely identifies each project
`name` | The title of the project
`description` | A description of the project, the school, etc.
`needStatement` | The specific requirements to accomplish the project
`category` | The area or subject the project fits into
`level` | The level of the classroom
`resource` | The type of requested resources
`cost` | The total amount ($) needed to meet the needs of the project
`status` | The current status of the proposal

### Create a Table
First of all, we have to create a new table, called `projects`, in the database.

You can also add the attributes. There are two parameters that need to be specified (see the documentation for a detailed explanation). The first, the type of attribute, is mandatory. The second, indexed, should only be used for those attributes that will be used to filter the projects.

In [ ]:
SherpaRequest(SherpaRequest.Verb.POST, "/v2/recomm/tables", data='{\"tableId\": \"projects\",\"engine\": \"content_based\", \"attributes\": [{\"attributeId\": \"name\",\"type\": \"string\"}, {\"attributeId\": \"description\",\"type\": \"string\"}, {\"attributeId\": \"needStatement\",\"type\": \"string\"}, {\"attributeId\": \"category\",\"type\": \"set\"}, {\"attributeId\": \"level\",\"type\": \"category\",\"indexed\": \"true\"}, {\"attributeId\": \"resource\",\"type\": \"category\",\"indexed\": \"true\"}, {\"attributeId\": \"cost\",\"type\": \"double\"}, {\"attributeId\": \"status\",\"type\": \"category\",\"indexed\": \"true\"} ]}').perform_request();

By default, when a new table is created, the system creates three attributes: `expiration`. We can check, by using the following command:

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/projects/attributes").perform_request();

### Add a New Item

Now we can add items to the catalog. Let us consider the following project:

Field | Value
--- | ---
itemId | 69da11b15b82cf59c389ba81c444731e
name | A Whole New World
description | Seeing my students learn is the thing that makes me the happiest. I love to be able to give them all the information that they need in order to understand and appreciate the world that they live in. Curious, creative, and very vocal, my students are the future of our society and I want to make sure they are equipped and ready to handle it. Faced with some difficult obstacles that they have to go through in their own neighborhood, I want them to know that the obstacles in the classroom can be overcome. With hard work and determination they will become the best of our future. My students truly love science. They enjoy the lessons that they do in class. An iPad can show them so much more about science. They would be able to watch videos about animals, see the different stages of matter and observe the life cycle of a plant. My students need to see first hand how science works in the world. I want them to see how science is all around them everyday by them focusing on the plants and trees, the sun and the water cycle. An iPad will show the students in real time how science affects us worldwide.
needStatement | My students need an iPad to help them learn more about science.
category | Math & Science
level | Grades PreK-2
resource | Computers & Tablets
cost | 374.64
status | Live

We can add it to the database by using the following command:

In [ ]:
SherpaRequest(SherpaRequest.Verb.POST, "/v2/recomm/projects/items", data='{\"itemId\": \"69da11b15b82cf59c389ba81c444731e\",\"name\": \"A Whole New World\",\"description\": \"Seeing my students learn is the thing that makes me the happiest. I love to be able to give them all the information that they need in order to understand and appreciate the world that they live in. Curious, creative, and very vocal, my students are the future of our society and I want to make sure they are equipped and ready to handle it. Faced with some difficult obstacles that they have to go through in their own neighborhood, I want them to know that the obstacles in the classroom can be overcome. With hard work and determination they will become the best of our future. My students truly love science. They enjoy the lessons that they do in class. An iPad can show them so much more about science. They would be able to watch videos about animals, see the different stages of matter and observe the life cycle of a plant. My students need to see first hand how science works in the world. I want them to see how science is all around them everyday by them focusing on the plants and trees, the sun and the water cycle. An iPad will show the students in real time how science affects us worldwide.\",\"needStatement\": \"My students need an iPad to help them learn more about science.\",\"category\": [\"Math & Science\"],\"level\": \"Grades PreK-2\",\"resource\": \"Computers & Tablets\",\"cost\": 374.64,\"status\": \"Live\"}').perform_request();

This operation can be done anytime we want to include a new item in the catalog. However, the API also has an endpoint to load massive data from a single file at once. Go to [Loading Massive Data](#Loading-Massive-Data) for a detailed example about this option.



The following `GET` command confirms that the item has been correctly stored in the database:

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/projects/items/69da11b15b82cf59c389ba81c444731e").perform_request();

### Define the User Profile
In this section, we will explain how to define the profile of the donors that will get recommendations about projects from DonorsChoose. In this example, we are going to work with a CSV file containing the following information about the donors:

Field | Description
--- | ---
userId | A string that uniquely identifies each donor
city | The donor's city of residence
state | The donor's state of residence
isTeacher | Whether the donor is a teacher or not
zipCode | The first three digits of the zip code of the donor's place of residence

### Create User Attributes
When adding new attributes, there are two parameters that need to be specified (see [documentation](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/api-reference/users/) for a detailed explanation), the same as for item attributes: `type` and `indexed`.

**Note**: There is no need to create the `userId` attribute, since it is managed internally by the API.

In this case, we will choose `city`, `state` and `isTeacher` as indexable attributes. The commands are the following:

In [ ]:
SherpaRequest(SherpaRequest.Verb.POST, "/v2/recomm/users/attributes", data='{\"attributes\": [{\"attributeId\": \"city\",\"type\": \"string\",\"indexed\": \"true\"},{\"attributeId\": \"state\",\"type\": \"category\",\"indexed\": \"true\"},{\"attributeId\": \"isTeacher\",\"type\": \"boolean\",\"indexed\": \"true\"},{\"attributeId\": \"zipCode\",\"type\": \"int\"}]}').perform_request();

To confirm that all attributes were added correctly, we can run the `GET` command:

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/users/attributes").perform_request();

### Add a New User
It is time to add a new user to the database. Let us consider the following donor:

Field | Value
--- | ---
userId | 5f24f7ece308e11c9e31a6b9ad53cf68
city | Hempstead
state | New York
isTeacher | Yes
zipCode | 115

We can add them to the database, by using the following command:

In [ ]:
SherpaRequest(SherpaRequest.Verb.POST, "/v2/recomm/users", data='{\"userId\": \"5f24f7ece308e11c9e31a6b9ad53cf68\",\"city\": \"Hempstead\",\"state\": \"New York\",\"isTeacher\": \"Yes\",\"zipCode\": 115}').perform_request();

Users can be created this way, or by means of an endpoint for massive data loading from a file. Go to [Loading Massive Data](#Loading-Massive-Data) for a detailed example about this option.

We can confirm that the donor has been correctly registered, using the following `GET` command:

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/users/5f24f7ece308e11c9e31a6b9ad53cf68").perform_request();

### Register Interactions

The basis of the recommender systems that the API provides is the set of interactions between users and items. In this section, we consider donations made by donors to the existing projects.

#### Create a Type of Interaction

First of all, the interaction itself needs to be defined. There are five parameters to customize it: `weight`, `cumulative`, `interactionType`, `maxValue`, and `minValue` (see the [documentation](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/api-reference/items/) for specific details about each of them). In this case, donations can be cumulative with a continuous nonnegative value. Thus, the command to create it is the following:

In [ ]:
SherpaRequest(SherpaRequest.Verb.POST, "/v2/recomm/projects/interactions", data='{\"interactionId\": \"donate\",\"type\": \"continuous\",\"cumulative\": true,\"weight\": 1.0,\"minValue\": 0.0}').perform_request();

**Note**: We arbitrarily assigned weight 1.0 to the interaction, since we only consider one interaction type. This parameter becomes relevant when two or more interaction types come into play.

A `GET` command can confirm that the insertion is correct. The response contains all the parameters, including those that we did not fix:

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/projects/interactions").perform_request();

### Register a New Interaction
Once the interaction is created, we can upload the donations to the projects. For example, let us consider the following:

Field | Value
--- | ---
userId | 5f24f7ece308e11c9e31a6b9ad53cf68
itemId | 69da11b15b82cf59c389ba81c444731e
interactionId | donate
timestamp | 2018-04-15 13:06:01
value | 25

**Note**: When inserting new interactions, timestamps need to be expressed as epoch times in milliseconds.

We can add it to the database by using the following command:

In [ ]:
SherpaRequest(SherpaRequest.Verb.POST, "/v2/recomm/projects/interactions/donate/5f24f7ece308e11c9e31a6b9ad53cf68/69da11b15b82cf59c389ba81c444731e", data='{\"timestamp\": 1523797561000,\"value\": 25}').perform_request();

To check that the previous command saved the data correctly, we can list the interactions made by that user. This command returns the unique interaction that the user has:

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/projects/users/5f24f7ece308e11c9e31a6b9ad53cf68/donate").perform_request();

It is also possible to check by listing the donations made to the project:

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/projects/items/69da11b15b82cf59c389ba81c444731e/donate").perform_request();

## Loading Massive Data
### Quickstart Guide
In this tutorial, we provide examples about loading files including multiple users, items, or interactions, the three cases being quite similar. As stated in [Building a Recommender System](#Building-a-Recommender-System), both users and items can be introduced, one by one, into the recommender system. However, it can be helpful to load multiple entities by using a single command. For instance, that would be the case when migrating an existing database.

In these examples, we assume that the structure has already been created (see [Building a Recommender System](#Building-a-Recommender-System) for step-by-step instructions). We have sampled three of the files from the dataset [Data Science for Good: DonorsChoose.org](https://www.kaggle.com/donorschoose/io), in order to simplify the examples:

- [`Projects.csv`](https://recommender-tutorial-data-set.s3-eu-west-1.amazonaws.com/Projects.csv): 12,804 projects (or items).
- [`Donors.csv`](https://recommender-tutorial-data-set.s3-eu-west-1.amazonaws.com/Donors.csv): 2,000 donors (or users).
- [`Donations.csv`](https://recommender-tutorial-data-set.s3-eu-west-1.amazonaws.com/Donations.csv): 4,975 donations (or interactions).

### File Formats
Currently, the Sherpa.ai Custom Content Recommendation API is capable of interpreting two file formats: JSON and CSV. You can choose the one that best fits your needs, as long as the contents are correctly structured:

Type of data | Mandatory fields | Optional fields | Observations
--- | --- | --- | ---
Items | `itemId` | Any item attribute | Item attributes have to be previously defined. Items have to be new; updates are not allowed.
Users | `userId` | Any user attribute | User attributes have to be previously defined. Users have to be new; updates are not allowed.
Interactions | `itemId`, `userId`, `interactionId`, `timestamp` | `value` | The interaction types have to be previously defined. User-item interactions have to be new; updates are not allowed.

### Register an Upload Order
First, a batch upload order is required for each of the files. The files `Projects.csv`, `Donors.csv` and `Donations.csv`, which are already uploaded on `https://recommender-tutorial-data-set.s3-eu-west-1.amazonaws.com/`, contain the data we want to upload. The JSONs returned in the response to these commands include two fields: `requestId` and `status`. The former is the needed to check the order status, which is indicated by the latter. We will keep a reference to the request paths and `requestId`s for the next steps, where we will check the upload status.

In [ ]:
batchUploadRequests = {}
response = SherpaRequest(SherpaRequest.Verb.POST, "/v2/recomm/projects/items/batch", data='{\"url\": \"https://recommender-tutorial-data-set.s3-eu-west-1.amazonaws.com/Projects.csv\",\"format\": \"csv\"}').perform_request();
batchUploadRequests["/v2/recomm/projects/items/batch"] = json.loads(response.text)["requestId"]
response = SherpaRequest(SherpaRequest.Verb.POST, "/v2/recomm/users/batch", data='{\"url\": \"https://recommender-tutorial-data-set.s3-eu-west-1.amazonaws.com/Donors.csv\",\"format\": \"csv\"}').perform_request();
batchUploadRequests["/v2/recomm/users/batch"] = json.loads(response.text)["requestId"]
response = SherpaRequest(SherpaRequest.Verb.POST, "/v2/recomm/projects/interactions/batch", data='{\"url\": \"https://recommender-tutorial-data-set.s3-eu-west-1.amazonaws.com/Donations.csv\",\"format\": \"csv\"}').perform_request();
batchUploadRequests["/v2/recomm/projects/interactions/batch"] = json.loads(response.text)["requestId"]

### Check Processing Status
The work orders are processed on-demand, but if those commands are run immediately after posting the orders, the response will be the same as above (`queued`). When the system starts to process the files, the status will change to `processing`.

In [ ]:
for path, requestId in batchUploadRequests.items():
    SherpaRequest(SherpaRequest.Verb.GET, path + "/" + requestId).perform_request();

Depending on the file sizes, the process can last anywhere from minutes to hours.

### Verify Completed Uploads

After the uploading operation finishes, if no fatal error has occurred, there are two possible statuses: `completed` and `completed with errors`. The former asserts that all the elements in the files were uploaded correctly, whereas the latter is returned when some of them could not be saved correctly. This last one will be the case for three of the files:

- The donations file includes the one saved in [Add a New Item](#Add-a-New-Item), so that duplicate is detected.
- The donor from [Add a New User](#Add-a-New-User) is also detected as a duplicate.
- The interactions file contains more errors. There are many duplicates (including the donation that had already been saved in [Register a New Interaction](#Register-a-New-Interaction)), due to the fact that there can only be one interaction per timestamp (with millisecond precision).

We will now wait until the datasets have been uploaded:

In [ ]:
processingStatuses = ["queued", "processing"]
processedRequestIds = {}
while len(processedRequestIds) != len(batchUploadRequests):
    for path, requestId in batchUploadRequests.items():
        if requestId not in processedRequestIds.keys():
            request = SherpaRequest(SherpaRequest.Verb.GET, path + "/" + requestId)
            response = request.perform_request(pretty_print=False);
            if json.loads(response.text)["status"] in processingStatuses:
                response = None
                print(".", end="")
                time.sleep(5)
            else:
                processedRequestIds[requestId] = response
                clear_output(wait=True)
                print("Processed " + str(len(processedRequestIds)) + " of " + str(len(batchUploadRequests)))
clear_output(wait=True)
for response in processedRequestIds.values():
    SherpaRequest.pretty_print_response(response=response)

## Training a Model
#### Quickstart Guide
In order to perform accurate recommendations, the model needs to be trained. The training can be performed on-demand at any time. At the moment, it's possible to train a model for the following engines: `hybrid` and `content_based`.

**Note**: For the correct model to be generated, the engine must match the one selected for recommendations when [Creating a Table](#Create-a-Table). Additional trainings might be performed for different engines, but they will have no effect on the recommendations of each other.

### On-demand training
In order to make recommendation using the `content_based` engine, we will train the model for the dataset we've loaded in [Loading Massive Data](#Loading-Massive-Data):

In [ ]:
response = SherpaRequest(SherpaRequest.Verb.POST, "/v2/recomm/projects/train", data='{\"engine\" : \"content_based\"}').perform_request();
trainingRequestId = json.loads(response.text)["requestId"]

We will now wait for the training to end:

In [ ]:
processingStatuses = ["queued", "processing"]
while trainingRequestId != None:
    request = SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/projects/train/" + trainingRequestId)
    response = request.perform_request(pretty_print=False);
    if json.loads(response.text)["status"] in processingStatuses:
        response = None
        print(".", end="")
        time.sleep(5)
    else:
        trainingRequestId = None
        break
clear_output(wait=True)
SherpaRequest.pretty_print_response(response=response) 

## Making Recommendations
#### Quickstart Guide

The Sherpa.ai Custom Content Recommendation API is meant to provide you with relevant recommendations based on the characteristics, tastes, and interactions of both users and items. The goal of this tutorial is to provide some insight into the options that the API offers.

The examples provided are based on two datasets. On the one hand, we will use [Data Science for Good: DonorsChoose.org](https://www.kaggle.com/donorschoose/io), regarding school projects and people that make donations to fund them (in particular, the files `Projects.csv`, `Donors.csv` and `Donations.csv` presented in [Building a Recommender System](#Building-a-Recommender-System)). On the other hand, we will use [MovieLens 100K](https://grouplens.org/datasets/movielens/100k/) and, more specifically, the files `u.item` (for movies), `u.user` (for users), and `u.data` (for ratings given to movies by users). In the following examples, we will assume that the whole of the datasets has been imported into the API. Please, refer to [Building a Recommender System](#Building-a-Recommender-System) and [Loading Massive Data](#Loading-Massive-Data) for step-by-step instructions.

### Choose a Recommender Engine

Building useful recommendations is the main objective of the API. To achieve this, it is very important to choose the recommender engine that best fits the characteristics of the dataset.

This selection can be made when creating the table (see [Building a Recommender System](#Create-a-Table)), and can also be changed at any time:

In [ ]:
SherpaRequest(SherpaRequest.Verb.PATCH, "/v2/recomm/tables/projects", data='{\"engine\": \"content_based\"}').perform_request();

The examples included in this tutorial cover the use of both engines. Since the [DonorsChoose.org](https://www.kaggle.com/donorschoose/io) dataset has some long text fields (such as `description` and `needStatement`), the most appropriate choice seems to be the [Content-based](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/how-it-works#recommendations) engine. However, the several numerical and categorical fields of the [MovieLens 100K](https://grouplens.org/datasets/movielens/100k/) dataset make it suitable for the [Hybrid](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/how-it-works#recommendations) engine.

**Note**: The API is meant to work with a unique set of users, regardless of the number of item catalogs. Therefore, we recommend deleting any existing datasets, before trying a new one.

### Content-based Recommender Engine

At this point, we have already created a catalog of projects from [DonorsChoose.org](https://www.kaggle.com/donorschoose/io) and saved information and donations made by many users. Thus, the system is ready to recommend new projects to donors. Let us recall that we have chosen the [Content-based recommender engine](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/how-it-works#recommendations) to build the recommendations.

There are two ways of recommending new items to users: general recommendations and filtered recommendations.

#### General Recommendations

Let us consider user `5f24f7ece308e11c9e31a6b9ad53cf68` [again](#Add-a-New-User). After the batch import done in [Loading Massive Data](#Loading-Massive-Data), we have 74 donations made by this user to 45 different projects.

The first and most general way of recommending simply consists of returning the most suitable projects, without any further restrictions:

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/projects/users/5f24f7ece308e11c9e31a6b9ad53cf68/items?limit=5").perform_request();

**Note**: The `limit` parameter is optional, but preferable. If it is not used, the API can return up to 500 items. For pagination, use the `afterId` parameter.

#### Filtered Recommendations

When making recommendations, we can also impose restrictions on the set of recommendable elements. Using [RSQL](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/api-reference/rsql), we can define conditions to be fulfilled by the items to be recommended. It is important for the attributes used here to be indexed, in order to obtain recommendations in an acceptable amount of time (refer to the [documentation](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/api-reference/items#add-item-attribute) for further details).

Let's consider user `5f24f7ece308e11c9e31a6b9ad53cf68` [again](#Add-a-New-User). After the batch import done in [Loading Massive Data](#Loading-Massive-Data), we have 74 donations made by this user to 45 different projects.

In our [Create a Catalog](#Create-a-Catalog) example, we defined three indexed attributes: `level`, `resource` and `status`. Let us consider, for instance, projects of "Grades 3-5" level.

The response obviously contains the project shown above; if it is interesting without restrictions, it is even more so if it satisfies them. But the response includes other recommendations:

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/projects/users/5f24f7ece308e11c9e31a6b9ad53cf68/items?limit=5&filter=level==\"Grades PreK-2\"").perform_request();

#### Delete [DonorsChoose.org](https://www.kaggle.com/donorschoose/io) example's data and setup
The API is meant to work with a unique set of users, regardless of the number of item catalogs. Therefore, we recommend deleting any existing datasets, before trying a new one.

Run the cell below to remove the data and the setup:

In [ ]:
SherpaRequest(SherpaRequest.Verb.DELETE, "/v2/recomm/tables/projects").perform_request();
SherpaRequest(SherpaRequest.Verb.DELETE, "/v2/recomm/users").perform_request();
SherpaRequest(SherpaRequest.Verb.DELETE, "/v2/recomm/users/attributes/city").perform_request();
SherpaRequest(SherpaRequest.Verb.DELETE, "/v2/recomm/users/attributes/state").perform_request();
SherpaRequest(SherpaRequest.Verb.DELETE, "/v2/recomm/users/attributes/zipCode").perform_request();
SherpaRequest(SherpaRequest.Verb.DELETE, "/v2/recomm/users/attributes/isTeacher").perform_request();

### Hybrid Recommender Engine

For the examples below, we will assume that the catalog of movies from [MovieLens 100K](https://grouplens.org/datasets/movielens/100k/) is already created, ratings sent by users are already saved and the dataset has been trained. Thus, the system is ready to recommend new movies. Let us recall that we have chosen the [Hybrid Recommender Engine](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/how-it-works#recommendations), to build the recommendations.

Let us consider user 147, a 40-year-old, female librarian that lives within zip code 02143:

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/users/147").perform_request();

There are two ways of recommending new items to users: general recommendations and filtered recommendations. Both of them include a percent match score, a number between 0 and 100 that indicates the degree of affinity between the recommended entity and the recipient.

#### General Recommendations

The first and most general way of recommending simply consists of returning the most suitable movies, without any further restrictions. The response contains five movies which might be of interest to the user, taking into account the previous interactions.

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/movies/users/147/items?limit=5").perform_request();

**Note**: The `limit` parameter is optional, but preferable. If it is not used, the API can return up to 500 items. For pagination, use the `afterId` parameter.

#### Filtered Recommendations

When making recommendations, we can also impose restrictions on the set of recommendable elements. Using [RSQL](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/api-reference/rsql), we can define conditions to be fulfilled by the items to be recommended. It is important for the attributes used here to be indexed, in order to obtain recommendations in an acceptable amount of time (refer to the [documentation](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/api-reference/items#add-item-attribute) for further details).

In our [MovieLens 100K](https://grouplens.org/datasets/movielens/100k/) example, we have defined the year attribute to be indexed. So, let's imagine that user `147` would like to watch a movie, but an old one (from before 1970, for instance). These are the five movies recommended by the API:

In [ ]:
SherpaRequest(SherpaRequest.Verb.GET, "/v2/recomm/movies/users/147/items?limit=5&filter=year=le=1970").perform_request();

## Next steps
This is the end of the tutorial. You can now continue exploring the API by checking the [API Reference](https://developers.sherpa.ai/recommender/custom-content-recommendation-api/api-reference/introduction) of the [Sherpa developers](https://developers.sherpa.ai) site.

**Note**: The API is meant to work with a unique set of users, regardless of the number of item catalogs. Therefore, we recommend deleting any existing datasets, before trying a new one.

Thank you!